# Differential Equations Practicum
The purpose of this activity is to implement and/or understand the principles of numerical methods in differential equations.

Variant 24.

Original Equation:

$$
y' = xy^2-3xy,\:y(0)=2\\
\dfrac{y'}{y^2}=x-\dfrac{3x}{y},\:y!=0\\
z=1/y,\:dz=-\dfrac{dy}{y^2}\\
-z'=x(1-3z)\\
\dfrac{z'}{3z-1}=x\\
\dfrac{\ln{(3z-1)}}{3}=\dfrac{x^2}{2}+C_1\\
-1+3z=C_2e^{\dfrac{3x^2}{2}}\\
\dfrac{3}{y}=C_2e^{\frac{3x^2}{2}}+1\\
y=\dfrac{3}{1+C_2e^{\frac{3x^2}{2}}}\\
For\:y(0)=2\:=>\:2=\dfrac{3}{C_2+1}\:=>\:C_2=\frac{1}{2}
$$

Exact Solution:

$$
y=\frac{6}{2+e^{\frac{3x^2}{2}}}
$$



In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
import numpy
output_notebook()

Loading BokehJS ...

## Definition of original and exact solution functions

In [2]:
def my_func(x, y):
    return x * y * y - 3 * x * y

In [3]:
def exact_func(x,C):
    return 3/(C*numpy.exp(3*x*x/2)+1)

## Helper functions

In [4]:
def get_x(xs, xf, N):
    return numpy.linspace(xs, xf, num=N)

# Computation for the exact function
Receive initial value for x and y, end of the interval [x0,X] and number of steps to perform

In [5]:
def exact(func, x0, y0, X, N_steps):
    func = exact_func
    x = get_x(x0, X, N_steps)
    y = numpy.array(y0)
    # Compute the constant of the general solution
    C = (3/y0-1)/numpy.exp(3*x0*x0/2)
    
    # Compute ys and store them into the array
    # Loop until 50 elements (from 1 because we start with y0)
    for i in range(1, N_steps):
        y = numpy.append(y,func(x[i],C))
    return x, y

## Euler Method

In [6]:
# Get function, initial value for x and y, end of the interval [x0,X] and number of steps to perform
def euler(func, x0, y0, X, N_steps):
    # Resulting pairs of x, y will be stored in arrays
    x = get_x(x0, X, N_steps)
    y = numpy.array([y0])
    
    # Calculate the step for x
    h = (X - x0) / float(N_steps)
    
    # Loop until 50 elements (-1 because we start with y0)
    for i in range(N_steps-1):
        # Perform Euler method
        # y(i+1)=yi+h*f(xi,yi)
        y = numpy.append(y, y[i] + h * func(x[i], y[i]))
    return x, y

## Improved Euler Method

In [7]:
# Get function, initial value for x and y, end of the interval [x0,X] and number of steps to perform
def improved_euler(func, x0, y0, X, N_steps):
    # Resulting pairs of x, y will be stored in arrays
    x = get_x(x0, X, N_steps)
    y = numpy.array([y0])
    
    # Calculate the step for x
    h = (X - x0) / float(N_steps)

    # Loop until 50 elements (-1 because we start with y0)
    for i in range(N_steps-1):
        # Perform Improved Euler Method
        # y(i+1) = yi + h/2 * (f(xi,yi)+f(x(i+1),yi+h*f(xi,yi)))
        # where m1 = f(xi,yi) and m2 = f(x(i+1),yi+h*m1)
        m1 = func(x[i], y[i])
        m2 = func(x[i + 1], y[i] + h * m1)
        y = numpy.append(y, y[i] + (h * (m1 + m2)) / 2)
    return x, y

## Runge Kutta Method

In [8]:
# Get function, initial value for x and y, end of the interval [x,X0] and number of steps to perform
def runge_kutta(func, x0, y0, X, N_steps):
    # Resulting pairs of x, y will be stored in arrays  
    x = get_x(x0, X, N_steps)
    y = numpy.array([y0])
    
    # Calculate the step for x
    h = (X - x0) / float(N_steps)
    
    # Loop until 50 elements (-1 because we start with y0)
    for i in range(N_steps-1):
        # Perform Runge-Kutta method
        k1 = h * func(x[i], y[i])
        k2 = h * func(x[i] + h / 2, y[i] + k1 / 2)
        k3 = h * func(x[i] + h / 2, y[i] + k2 / 2)
        k4 = h * func(x[i] + h, y[i] + k3)
        y = numpy.append(y, y[i] + (k1 + 2 * k2 + 2 * k3 + k4) / 6)
    return x, y

## Computation of all methods

In [9]:
def all_methods(func, x0, y0, X, N_steps):
    x = get_x(x0, X, N_steps)
    _, y_exact = exact(func, x0, y0, X, N_steps) 
    _, y_euler = euler(func, x0, y0, X, N_steps)
    _, y_improved = improved_euler(func, x0, y0, X, N_steps)
    _, y_runge = runge_kutta(func, x0, y0, X, N_steps)
    return x, y_exact, y_euler, y_improved, y_runge

Create widgets for changable x0, y0, X and number of steps

In [10]:
x0_widget = widgets.FloatSlider(value=0, min=-100, max=100)
y0_widget = widgets.FloatSlider(value=2, min=-100, max=100)
X_widget = widgets.FloatSlider(value=6.4, min=-10, max=10)
N_steps_widget = widgets.IntSlider(value=50, min=45, max=200)
x0=0
y0=2
X=6.4
N_steps=50
x, y_exact, y_euler, y_improved, y_runge = all_methods(my_func, x0, y0, X, N_steps)

## Comparison of methods via plotting
We will compare different methods by plotting their graphs with the exact solution

In [11]:
methods_graph = figure(plot_width=400, plot_height=400, title="Methods Comparison", x_axis_label='x', y_axis_label='y')

exact_line = methods_graph.line(x, y_exact,legend='Exact solution', line_color='#000000')
euler_line = methods_graph.line(x, y_euler,legend='Euler method', line_color='#FF0000', alpha=0.5)
improved_line = methods_graph.line(x, y_improved,legend='Improved Euler method', line_color='#00FF00',alpha=0.5)
runge_line = methods_graph.line(x, y_runge,legend ='Runge-Kutta method', line_color='#0000FF', alpha=0.5)

# Error Graph
Comparison of error for all of the graphs

In [12]:
# y_error_euler = numpy.abs(y_exact-y_euler)
# y_error_improved = numpy.abs(y_exact-y_improved)
# y_error_runge = numpy.abs(y_exact-y_runge)

# error_1 = figure(plot_width=400, plot_height=400, title="Euler")
# error_2 = figure(plot_width=400, plot_height=400, title="Improved Euler")
# error_3 = figure(plot_width=400, plot_height=400, title="Runge Kutta")

# error1_r = error_1.line(x, y_error_euler, color='red', line_width=2)
# error2_r = error_2.line(x, y_error_improved, color='red', line_width=2)
# error3_r = error_3.line(x, y_error_runge, color='red', line_width=2)

In [13]:
def update_plot(x0=0, y0=2, X=6.4,N_steps=50):
    if X <= x0:
        return
    if y0 == 0:
        return
    x, y_exact, y_euler, y_improved, y_runge = all_methods(my_func, x0, y0, X, N_steps)
    
    exact = {'x': x, 'y': y_exact}
    euler = {'x': x, 'y': y_euler}
    improved = {'x': x, 'y': y_improved}
    runge = {'x': x, 'y': y_runge}

#     ex_er = {'x': x,'y': numpy.abs(y_exact-y_euler)}
#     ex_er = {'x': x,'y': numpy.abs(y_exact-y_improved)}
#     ex_er = {'x': x,'y': numpy.abs(y_exact-y_runge)}

    exact_line.data_source.data = exact
    euler_line.data_source.data = euler
    improved_line.data_source.data = improved
    runge_line.data_source.data = runge

    push_notebook(m_handle)

In [14]:
m_handle = show(methods_graph, notebook_handle=True)

In [15]:
interact(update_plot, x0=x0_widget, y0=y0_widget, X=X_widget,N_steps=N_steps_widget)

interactive(children=(FloatSlider(value=0.0, description='x0', min=-100.0), FloatSlider(value=2.0, description…

<function __main__.update_plot(x0=0, y0=2, X=6.4, N_steps=50)>

In [16]:
# show(error_1, notebook_handle=True)
# show(error_2, notebook_handle=True)
# show(error_3, notebook_handle=True)